In [1]:
import json
from datasets import Dataset, Audio

audio = Audio(sampling_rate = 16000)

In [2]:
with open('slice-0.25.json') as fopen:
    d = json.load(fopen)
    
filtered = []
for d_ in d:
    labels = d_['labels']
    scores = d_['scores']
    if len(labels) == 1 and labels[0] == 'Speech' and scores[0] < -1.0:
        continue
    if len(labels) > 1:
        filtered_labels, filtered_scores = [], []
        for no, l in enumerate(labels):
            if l == 'Speech' and scores[no] < 1.0:
                continue
            filtered_labels.append(l)
            filtered_scores.append(scores[no])
        d_['labels'] = filtered_labels
        d_['scores'] = filtered_scores
    filtered.append(d_)
    
len(filtered), len(d)

(794023, 939264)

In [3]:
dataset = Dataset.from_list(filtered)

In [4]:
dataset = dataset.cast_column("audio_filename", audio)
dataset

Dataset({
    features: ['audio_filename', 'labels', 'scores'],
    num_rows: 794023
})

In [7]:
dataset.push_to_hub('mesolitica/AudioSet-Chunk', split = 'chunk_0.25s')

In [8]:
from collections import defaultdict
import pandas as pd

df = pd.read_csv('class_labels_indices.csv')
mapping = {}
labels = {}
rev_labels = {}
counts = defaultdict(int)
for i in range(len(df)):
    mapping[df['mid'].iloc[i]] = df['index'].iloc[i]
    labels[df['index'].iloc[i]] = df['display_name'].iloc[i]
    rev_labels[df['display_name'].iloc[i]] = df['index'].iloc[i]
    counts[df['index'].iloc[i]] = 0

In [9]:
for d in filtered:
    for l in d['labels']:
        counts[rev_labels[l]] += 1
        
for k, v in counts.items():
    print(k, labels[k], ':', v)

0 Speech : 97017
1 Male speech, man speaking : 1508
2 Female speech, woman speaking : 153
3 Child speech, kid speaking : 485
4 Conversation : 0
5 Narration, monologue : 1576
6 Babbling : 18
7 Speech synthesizer : 159
8 Shout : 0
9 Bellow : 192
10 Whoop : 571
11 Yell : 117
12 Battle cry : 689
13 Children shouting : 279
14 Screaming : 207
15 Whispering : 282
16 Laughter : 721
17 Baby laughter : 232
18 Giggle : 474
19 Snicker : 674
20 Belly laugh : 54
21 Chuckle, chortle : 507
22 Crying, sobbing : 74
23 Baby cry, infant cry : 549
24 Whimper : 131
25 Wail, moan : 165
26 Sigh : 370
27 Singing : 10724
28 Choir : 2469
29 Yodeling : 115
30 Chant : 749
31 Mantra : 1165
32 Male singing : 501
33 Female singing : 269
34 Child singing : 183
35 Synthetic singing : 50
36 Rapping : 154
37 Humming : 292
38 Groan : 180
39 Grunt : 1014
40 Whistling : 1170
41 Breathing : 14
42 Wheeze : 41
43 Snoring : 224
44 Gasp : 956
45 Pant : 380
46 Snort : 476
47 Cough : 125
48 Throat clearing : 903
49 Sneeze : 264
50